# Парсинг данных eLibrary для проекта "Паттерны коллаборации в Российском социологическом сообществе" - id авторов, 5.10.2022

В этом файле производится выгрузка данных eLibrary для проекта Лаборатории по гранту РНФ "Паттерны коллаборации в Российском социологическом сообществе" . Осуществляется импорт необходимых библиотек, загружаются id авторов, данные о которых необходимо выгрузить, производится выгрузка данных из API Elibrary.

Основной код выгрузки к API-011 Elibrary был подготовлен Ликой Капустиной в сентябре-октябре 2022 года. Цель выгрузки - выгрузить данные о статьях всех уникальных авторов, встречающихся в основном массиве данных (но не все данные по статьям).

**Результат выгрузки:**
1. Файл `all_final_authors_with_papers_5_10.csv`, содержащие в себе информацию о всех статьях всех авторов, присутствовавших в 8 колонках наших изначальных данных.
2. Файлы `all_final_authors_with_papers_5_10_only_after_2010.xlsx` и `all_final_authors_with_papers_5_10_only_after_2010.csv`, содержащие в себе информацию о тех статьях всех авторов (которые изначально присутствовали в 8 колонках наших данных), которые были написаны в 2010 году и после.

## Подготовка к работе

### Загрузка необходимых библиотек

In [2]:
#pip install xmltodict
from numpy import fromstring # приведение данных к строке
import numpy as np
import requests # модуль для запросов к данным по ссылкам
import xmltodict # работа с xml
import pandas as pd # модуль для работы с таблицами
import tqdm # модуль для работы с tqdm
from time import sleep # импортируем функцию сна 
import warnings # модуль с предупреждениями
warnings.filterwarnings("ignore") # выбираем игнорировать предупреждения
import xml.etree.ElementTree as ET # работа с xml
from xml.etree.ElementTree import parse # работа с xml
from urllib.request import urlopen # работа с запросами
import pickle # формат данных pickle

### Проверка ip-адреса

Выгрузка данных могла производиться только по рабочему ip-адресу Дарьи Васильевны в корпусе Вышки на Покровском бульваре, поэтому перед выгрузкой данных осуществлялось подключение к ip-адресу Дарьи Васильевны.

In [3]:
import http.client # модуль для работы с ip
conn = http.client.HTTPConnection("ifconfig.me")
conn.request("GET", "/ip")
conn.getresponse().read() # ip-адрес

b'92.242.58.41'

### Загрузка датафрейма с данными по публикациям

Сбор данных с помощью сервиса 011 в API Elibrary собрать информацию об авторах с ID (важно собрать информацию об основной рубрике по кодификатору ГРНТИ

In [8]:
df = pd.read_csv('FINAL_CORRECT_SOCIOLOGISTS.csv')

In [9]:
author_ids = [] # создаем пустой список

for i in range(1, 8+1):
    author_ids.extend(list(df[f'author_{i}_id'])) # расширяем наш список за счет всех 8 колонок с авторами

In [10]:
print(len(author_ids)) # всего собрано id авторов
author_ids = list(np.unique(author_ids))
print(len(author_ids)) # id уникальных авторов по всем 8 колонкам

601856
24983


In [11]:
author_ids[0:5] # так выглядят наши данные, их нужно обработать

['0', '0.0', '1000010', '1000056', '1000056.0']

In [12]:
for i in author_ids:
    print(i.replace('.0', ''))

0
0
1000010
1000056
1000056
1000082
1000082
1000152
1000176
100030
100032
1000501
1000613
1000712
1000736
10008
1000914
1000979
1000989
1001157
1001178
1001287
1001500
1001528
1001562
1001562
1001636
1001719
1001970
1001988
1001998
1002023
1002069
1002090
1002305
1002395
1002405
1002405
1002621
1002676
1002717
1002717
1002738
1002965
1002966
1002993
1003065
1003065
1003106
1003106
1003121
1003243
1003339
1003357
100348
1003590
1003703
1003781
100381
1003816
1003844
1003872
1003872
1003881
1003938
1003954
1003954
1003978
1004002
1004088
1004317
1004339
1004343
1004343
100438
1004380
1004390
1004400
1004407
1004489
1004563
1004599
1004627
1004634
1004838
1004858
1004909
1004914
1005191
1005291
1005323
1005336
1005336
1005403
100542
100542
1005495
1005497
1005538
1005587
1005587
1005755
1005879
1005879
1005960
1005971
100605
100605
1006137
1006185
1006235
1006235
1006295
1006295
1006430
1006430
1006449
1006569
1006582
1006619
1006619
1006939
1006946
1007008
1007027
1007114
1007152
1007176

1076401
1076474
1076513
107660
1076695
107676
1076893
1076893
1076905
1076907
107702
1077042
1077131
107714
107714
1077180
1077184
1077341
1077341
1077566
1077566
1077568
1077588
1077597
1077597
1077687
107776
1077766
1077798
1077848
1077872
1077878
1077884
1077972
1077986
1078012
1078014
1078014
1078016
1078028
107810
1078109
1078132
107825
107825
1078252
1078252
1078546
1078546
1078600
1078767
1078767
1078791
1078896
1079093
1079093
1079111
1079130
1079154
1079246
1079250
1079314
1079316
1079365
1079365
1079658
1079704
1079764
1079821
1079836
1079838
1079838
1080046
1080108
108025
1080328
1080335
1080375
1080498
1080515
1080577
1080689
1080777
1080838
1080838
1080900
1080906
1080944
1080965
1080965
1081007
1081041
1081046
1081137
108114
1081178
1081289
1081289
1081334
108140
1081445
108150
108150
1081705
1081709
1081713
1081713
108175
108176
1081770
1081880
1081922
1081930
1081930
1081961
1082002
1082002
1082036
1082056
1082279
1082299
1082299
108232
1082387
1082387
1082495
1082608
1

225688
226095
226095
226385
226385
226448
226476
226476
226639
226833
227165
227165
227275
227420
227420
227580
22769
227772
227772
227811
227843
227843
227916
228105
228350
228364
228390
228390
228631
228832
228930
229100
229100
229162
229162
229203
229212
229255
229418
229422
229422
229661
229661
22974
229757
22976
230206
230241
230311
230762
230762
230789
230825
230825
230841
230974
231032
231054
231104
231130
231442
231442
231523
231523
231849
231975
232117
232245
232245
232326
232512
232769
232769
232779
232831
233127
233154
233237
233867
233867
233875
234154
234378
234420
234420
234638
234653
234669
234976
235131
235131
235258
235258
235318
235554
235691
2357
235714
235741
235796
236157
236157
236165
236181
236181
23625
236324
236368
236368
236534
236689
236776
236797
236838
236838
237017
237017
237477
237477
237647
237678
237696
237704
237902
238362
238455
238512
239169
239185
239185
239388
239717
239729
239791
239791
239969
240000
240056
240689
240696
240729
240731
240731
24098

312942
312942
312972
313094
313315
313351
313353
313353
313531
313569
313580
313715
313716
313767
313767
313804
313804
313830
313894
313908
313908
313929
313998
313998
314059
314080
314123
314241
314244
314264
314288
314414
314582
314750
314776
314776
314792
314842
314842
314859
314859
314993
315123
315386
315410
315435
315435
315471
315478
315479
315566
315590
315590
315598
315625
315625
315680
315706
315725
315880
315907
315907
315978
316017
316017
316066
316082
316082
316133
316133
316191
316191
316275
316277
316277
316461
316461
316587
316587
316651
316700
316700
316727
316879
316904
316913
316953
316953
317061
317068
317095
317284
317339
317339
317389
317405
317413
317460
317460
317461
317461
317467
317483
317483
317491
317508
317510
317510
317556
317556
317567
317567
317568
317569
317569
317589
31765
317688
317688
317972
318132
318142
318166
318205
318205
318232
318348
318368
318397
318438
318467
318560
318613
318613
318705
318760
318904
318904
318964
319114
319170
319209
319281


363849
363875
363904
364003
364056
364070
364089
364090
364090
364100
364100
364132
364175
364217
364313
364459
364516
364525
364549
364708
364846
364913
364954
365064
365082
365082
365224
365260
365260
365275
365312
365312
365329
365353
365361
365515
365515
36558
365706
365748
365761
365860
365860
365887
365911
365973
365982
365982
366178
366209
366209
366215
366347
366383
366440
366440
366445
366477
366496
366546
366583
366603
366610
366610
366686
366823
367020
367020
367090
367114
367144
367216
367224
367236
367236
367328
367328
367398
367398
367435
367567
367570
367608
367835
368245
368279
368360
368362
368499
368499
368654
368671
368689
368716
36879
368792
368797
368797
368821
368821
368835
368835
368933
368933
368961
369028
369028
369102
369102
369118
369154
369173
369204
369241
369297
369413
369413
369467
369467
369489
3695
369575
369645
369645
369717
369717
369727
369737
369759
369789
369873
369878
369881
369881
369953
369979
370126
370204
370222
370249
370287
370288
370418
370

...

740374
740374
740385
740395
740444
740472
740530
74054
74054
740555
740568
740568
740577
740577
740704
740704
740748
740765
740771
740807
740816
740816
740825
740841
740841
74085
740882
740882
740913
740917
740917
74093
740931
740943
74095
741118
741141
741141
741202
741207
741319
741322
741353
74137
741388
741388
741448
74145
74147
741503
741512
741512
741524
741530
741543
741543
741575
741590
741633
741677
741677
741713
741728
74173
74173
741879
741911
741924
741924
741926
741959
741959
74199
742008
742079
742086
742099
742133
742150
742266
742267
742272
742297
742297
742315
742315
742324
742329
742329
742335
742335
742371
742390
742390
742392
742397
742397
74243
74243
742444
74245
74245
74246
74246
74247
74247
742476
742490
742511
742511
742531
742550
742550
742554
74258
74259
742603
742726
74274
742744
742780
742780
742794
74282
742824
74284
74284
74285
742880
742881
742881
74292
742964
742969
74299
74299
742998
74301
74301
74302
74308
74308
74310
743107
74311
74311
743124
74313
74

773291
773297
773297
77331
77331
773367
773402
773500
773500
77351
77355
77357
77357
77363
77363
773639
773651
773670
773672
773691
773691
773699
77371
77377
773854
773941
773946
773985
77400
774004
774008
77401
77401
774056
774070
774093
774096
77410
77410
77411
77411
774118
77412
77412
774122
774128
774128
774137
774137
774157
774157
774246
77426
774286
774286
774345
774361
774361
774366
774366
774371
774393
774421
774422
774422
774552
774552
774566
774566
774591
774693
774693
774699
774726
774785
774818
774818
774849
774883
774899
774899
77490
77490
774910
774910
774976
774976
77499
775022
77509
77510
775119
775161
775251
775251
775304
775323
775397
775397
775450
775458
775462
775502
775507
775564
775564
775622
775638
77564
775667
775680
775684
775724
775734
775739
775739
775740
775740
775763
775763
775942
775942
775953
775961
775969
775998
776033
776107
776125
776131
776132
776175
776181
776191
776206
776211
776218
776234
776234
776243
77640
776414
776443
776602
776602
776612
77661

826653
826653
826735
826794
826808
826830
826873
826875
826888
826891
826990
827043
827043
827100
827191
827191
827459
827459
827465
827482
827521
827545
827558
827573
827594
827594
827637
827656
827681
827704
827819
827867
827884
827891
827891
827921
827921
827959
828057
828057
828085
828220
828220
828225
828235
828235
828237
828237
828325
828406
828406
828416
828416
828463
828535
828546
828624
828728
828814
828880
828944
828973
828997
829011
829011
829165
829305
829425
829425
829438
829529
829597
829792
829823
829918
829930
829985
830042
830111
830146
830196
830224
830286
830286
830311
830311
830345
830444
830549
830549
830565
830716
830787
830812
83089
830899
830899
830901
830953
831000
83102
831046
831054
831061
831094
831149
831183
831215
831228
831245
831250
831357
831357
831474
831474
831538
831551
831551
831663
831791
831796
831796
831906
832076
832104
832122
832170
832188
832200
832202
832302
832328
832407
832442
832442
832478
832506
832660
832677
832698
832705
832705
832779
8

898011
898018
898241
898381
898386
898430
898468
898495
8985
898558
898611
898611
898631
898631
898664
898702
898705
898745
898757
898757
898759
898769
898779
898802
89883
898972
898990
899095
899095
899161
899195
899283
899502
899661
899982
900074
900074
900101
90016
900175
900316
900316
900330
900517
900605
900672
900698
900777
900905
900923
900969
901011
901011
901041
901136
901155
901157
901197
901202
901318
901323
901356
901356
901485
901485
901519
901563
901623
901709
901765
901859
902023
902278
902278
902295
902295
902377
902398
902437
902493
902497
902497
902612
902612
902633
902633
902772
902811
902833
902858
902870
902899
902918
902923
902992
902995
902995
903044
903092
903092
903096
903136
903188
903238
903242
903246
903270
903385
903385
903566
903566
903570
903593
903593
903638
903811
903840
903846
903846
903989
904
904050
904201
904242
904250
904331
904340
904340
904421
904439
904466
904479
904479
904574
904587
904638
904796
904896
905051
905093
905108
905108
905137
905179

982578
982578
982695
982737
98284
982896
982947
982948
982971
983075
983077
983077
983346
983346
983422
983570
983629
983630
983705
983759
983805
983805
983855
983898
983976
983985
983992
984054
984139
984327
98437
984508
984622
984680
984743
984756
984796
984848
984918
984925
984946
984950
984970
984970
984991
985033
985095
985219
985219
985390
985414
985668
985696
986008
986014
986069
986073
986221
986316
986316
986376
986376
986439
98651
986532
986537
986582
986649
986652
986756
986951
986953
986971
986986
987116
987145
987145
987176
987188
987270
987277
987306
987334
98742
987477
987559
98768
987789
987844
987844
987866
987880
987880
988057
988166
988179
988184
988184
988210
988210
988232
988244
988244
988293
988402
988402
988485
988485
98852
988785
988890
989018
989116
989141
989164
989353
989390
989472
989480
989517
989517
989521
989616
98982
98982
989961
989985
990041
990295
99035
990426
990461
990461
990754
990774
990953
990958
990959
990961
990997
991042
991055
991224
991376
9

In [13]:
author_ids.remove('nan')
author_ids.remove('none')
author_ids = [int(i.replace('.0', '')) for i in author_ids]
author_ids.remove(0)
author_ids.remove(0)
author_ids = list(np.unique(author_ids))

Получаем список с id авторов. id были собраны по всем 8 колонкам `author_{}_id`, удалены значения `0`, `nan`, `none`, значения обновлены до уникальных.

In [9]:
author_ids

[296,
 346,
 571,
 619,
 651,
 904,
 1269,
 1270,
 1277,
 1278,
 1282,
 1286,
 1293,
 1305,
 1319,
 1321,
 1339,
 1342,
 1354,
 1355,
 1358,
 1360,
 1361,
 1367,
 1375,
 1382,
 1386,
 1389,
 1393,
 1394,
 1395,
 1400,
 1404,
 1405,
 1426,
 1435,
 1438,
 1440,
 1444,
 1457,
 1461,
 1475,
 1478,
 1481,
 1484,
 1492,
 1611,
 1804,
 1805,
 1816,
 2357,
 2416,
 2575,
 2902,
 3238,
 3260,
 3370,
 3416,
 3470,
 3516,
 3527,
 3603,
 3695,
 3734,
 3837,
 4141,
 4163,
 4196,
 4213,
 4415,
 4464,
 5380,
 5439,
 5514,
 5559,
 6691,
 6740,
 7024,
 7574,
 7814,
 7967,
 8178,
 8195,
 8480,
 8523,
 8985,
 9157,
 9492,
 10008,
 10087,
 10116,
 10356,
 10453,
 11132,
 11203,
 11332,
 11442,
 11722,
 11783,
 12123,
 12225,
 12294,
 12312,
 12680,
 12928,
 12980,
 13257,
 13568,
 13692,
 13812,
 14072,
 14473,
 14699,
 14719,
 15140,
 15620,
 15671,
 15700,
 15886,
 16035,
 16122,
 16423,
 16438,
 16491,
 16570,
 16571,
 16924,
 16925,
 17068,
 17165,
 17220,
 17502,
 17572,
 17667,
 17789,
 18072,
 18657

In [14]:
len(author_ids) # количество уникальных id авторов по всем восьми колонкам

19723

**Пример тестовой выгрузки**: пробую выгрузку из сервиса 011. Произвожу тестовую выгрузку. Определила sid как 011 (сервис 011 для получения данных по авторам), в качестве аргумента нужно

In [25]:
# Сейчас просто выгружаем данные!
counter = 0
test_id = ['296'] # случайный id для теста. Обязательно нужно заключить в скобки, иначе данные не выгрузятся
for i in test_id:
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid="+i
    response = requests.get(url)
    data = xmltodict.parse(response.content)
    tree_response = urlopen(url).read()
    tree_data = ET.fromstring(tree_response)


In [26]:
data # вот так выглядит объект data

OrderedDict([('root',
              OrderedDict([('request',
                            OrderedDict([('remoteAddr', '92.242.58.41'),
                                         ('authorid', '296')])),
                           ('items',
                            OrderedDict([('@numOfItems', '252'),
                                         ('@limit', '683000'),
                                         ('item',
                                          [OrderedDict([('@id', '30817242'),
                                                        ('@typeCode', 'RAR'),
                                                        ('@yearPubl', '1970'),
                                                        ('@retracted', '0'),
                                                        ('#text',
                                                         '30817242')]),
                                           OrderedDict([('@id', '39931825'),
                                                        ('@t

In [27]:
data['root'] # так выглядит объект root в data

OrderedDict([('request',
              OrderedDict([('remoteAddr', '92.242.58.41'),
                           ('authorid', '296')])),
             ('items',
              OrderedDict([('@numOfItems', '252'),
                           ('@limit', '683000'),
                           ('item',
                            [OrderedDict([('@id', '30817242'),
                                          ('@typeCode', 'RAR'),
                                          ('@yearPubl', '1970'),
                                          ('@retracted', '0'),
                                          ('#text', '30817242')]),
                             OrderedDict([('@id', '39931825'),
                                          ('@typeCode', 'COA'),
                                          ('@yearPubl', '1975'),
                                          ('@retracted', '0'),
                                          ('#text', '39931825')]),
                             OrderedDict([('@id', '40082461'),

In [28]:
data['root']['items'] # так выглядит часть root в items

OrderedDict([('@numOfItems', '252'),
             ('@limit', '683000'),
             ('item',
              [OrderedDict([('@id', '30817242'),
                            ('@typeCode', 'RAR'),
                            ('@yearPubl', '1970'),
                            ('@retracted', '0'),
                            ('#text', '30817242')]),
               OrderedDict([('@id', '39931825'),
                            ('@typeCode', 'COA'),
                            ('@yearPubl', '1975'),
                            ('@retracted', '0'),
                            ('#text', '39931825')]),
               OrderedDict([('@id', '40082461'),
                            ('@typeCode', 'COA'),
                            ('@yearPubl', '1980'),
                            ('@retracted', '0'),
                            ('#text', '40082461')]),
               OrderedDict([('@id', '30973257'),
                            ('@typeCode', 'RAR'),
                            ('@yearPubl', '1981'),


In [29]:
# продолжаем уточнять
data['root']['items']['item'] # так выглядят итоговые объекты в которые забиты статьи одного автора

[OrderedDict([('@id', '30817242'),
              ('@typeCode', 'RAR'),
              ('@yearPubl', '1970'),
              ('@retracted', '0'),
              ('#text', '30817242')]),
 OrderedDict([('@id', '39931825'),
              ('@typeCode', 'COA'),
              ('@yearPubl', '1975'),
              ('@retracted', '0'),
              ('#text', '39931825')]),
 OrderedDict([('@id', '40082461'),
              ('@typeCode', 'COA'),
              ('@yearPubl', '1980'),
              ('@retracted', '0'),
              ('#text', '40082461')]),
 OrderedDict([('@id', '30973257'),
              ('@typeCode', 'RAR'),
              ('@yearPubl', '1981'),
              ('@retracted', '0'),
              ('#text', '30973257')]),
 OrderedDict([('@id', '30812715'),
              ('@typeCode', 'RAR'),
              ('@yearPubl', '1982'),
              ('@retracted', '0'),
              ('#text', '30812715')]),
 OrderedDict([('@id', '22760050'),
              ('@typeCode', 'PRC'),
              ('@ye

In [30]:
data['root']['items']['item'][0] # далее с помощью обычной индексаации можно получить каждую из статей.
# в дальнейшем будем пользоваться циклом for

OrderedDict([('@id', '30817242'),
             ('@typeCode', 'RAR'),
             ('@yearPubl', '1970'),
             ('@retracted', '0'),
             ('#text', '30817242')])

In [31]:
data['root']['items']['item'][0]

OrderedDict([('@id', '30817242'),
             ('@typeCode', 'RAR'),
             ('@yearPubl', '1970'),
             ('@retracted', '0'),
             ('#text', '30817242')])

In [33]:
for i in range(len(data['root']['items']['item'])):
    print(data['root']['items']['item'][i]) # вот такие словари у нас получаются. Это все статьи автора

OrderedDict([('@id', '30817242'), ('@typeCode', 'RAR'), ('@yearPubl', '1970'), ('@retracted', '0'), ('#text', '30817242')])
OrderedDict([('@id', '39931825'), ('@typeCode', 'COA'), ('@yearPubl', '1975'), ('@retracted', '0'), ('#text', '39931825')])
OrderedDict([('@id', '40082461'), ('@typeCode', 'COA'), ('@yearPubl', '1980'), ('@retracted', '0'), ('#text', '40082461')])
OrderedDict([('@id', '30973257'), ('@typeCode', 'RAR'), ('@yearPubl', '1981'), ('@retracted', '0'), ('#text', '30973257')])
OrderedDict([('@id', '30812715'), ('@typeCode', 'RAR'), ('@yearPubl', '1982'), ('@retracted', '0'), ('#text', '30812715')])
OrderedDict([('@id', '22760050'), ('@typeCode', 'PRC'), ('@yearPubl', '1986'), ('@retracted', '0'), ('#text', '22760050')])
OrderedDict([('@id', '31020309'), ('@typeCode', 'RAR'), ('@yearPubl', '1988'), ('@retracted', '0'), ('#text', '31020309')])
OrderedDict([('@id', '22472030'), ('@typeCode', 'RAR'), ('@yearPubl', '1989'), ('@retracted', '0'), ('#text', '22472030')])
OrderedD

In [34]:
for j in data['root']['items']['item'][0]: # берем первую статью
    print(j) # вот какие объекты забиты внутри одной записи про одну статью

@id
@typeCode
@yearPubl
@retracted
#text


In [35]:
print(data['root']['items']['item'][0]['@id']) # так получается id одной статьи
print(data['root']['items']['item'][0]['@typeCode']) # так получается тип
print(data['root']['items']['item'][0]['@yearPubl']) # так получается год
print(data['root']['items']['item'][0]['@retracted']) # так получается статус (отозвана?)
print(data['root']['items']['item'][0]['#text']) # так получается текст

30817242
RAR
1970
0
30817242


## Тестовая выгрузка с получением всех данных

Перед тем, как начать тестовую выгрузку, проверяю структуру данных из конца списка - беру последний айдишник автора и открываю его страницу. Я предварительно смотрела на тестовую выгрузку из пяти авторов из начала списка, и меня очень смутило количество их публикаций – сотни штук. Проверив автора с id `1158945`, я успокоилась: у него всего две публикации, за 2018 и 2022 год. Логиично, что у авторов из начала будет больше публикаций – они как минимум старше и раньше начали свою академическую деятельность.

На всякий случай, я перепроверила данные вручную. Алгоритм следующий:
1. Открывааем [Elibrary](https://www.elibrary.ru);
2. Входим в свой аккаунт;
3. В правом меню снизу выбираем `Навигатор` -> `Авторы`;
4. Открывается поисковое окно. В маленьком окошке выбираем `РИНЦ Author id` и вписываем в окошко справа значение id. Нажимаем на `Поиск`;
5. Нам показывают автора. Нажимаем на публикации (число публикаций). Открывается окно со всеми статьями, включенными в РИНЦ. Нажимаем на первую статью автора, далее - на имя и выбираем `Профиль автора в SCIENCE INDEX`;
6. Открывается страница со всеми публикациями автора в Elibrary. 

Для автора с id `1158945` открывается в результате манипуляций выше следующая ссылка: https://www.elibrary.ru/author_items.asp?authorid=1158945&show_option=1&show_refs=1 , на которой действительно находится две статьи - за 2018 и 2022 год.

In [49]:
author_ids[-1] # проверяем на авторе из конца списка
# ссылка для просмотра страницы
print(f'https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid={author_ids[-1]}')

https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=1158945


Тестовые ссылки:

1. Автор 296: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=296



2. Автор 346: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=346



3. Автор 571: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=571



4. Автор 619: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=619



5. Автор 651: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=651



In [62]:
counter = 0
test_id = author_ids[0:5] # 5 первых id для теста

one_row_one_paper = pd.DataFrame()

for j in tqdm.tqdm(test_id):
    author_id = j
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid="+str(j)
    response = requests.get(url)
    data = xmltodict.parse(response.content) # с этим объектом будем работать

    # создаем объекты с которыми будем работать, это все для записи данных по одному автору
    author_paper_ids = [] # айдишники статей одного автора
    author_types = [] # тип статей одного автора
    author_years = [] # год статей одного автора
    author_retracteds = [] # статус (отозвана или нет(?)) одного автора
    author_texts = [] # текст чего-то там одного автора
    
    # идем по всем статьям автора
    for i in range(len(data['root']['items']['item'])):
        
        ## ID статьи
        try:
            this_paper_id = data['root']['items']['item'][i]['@id']
        except:
            this_paper_id = 'none'
        author_paper_ids.append(this_paper_id) # добавляем id статьи к списку всех статей автора
        
        ## ТИП статьи 
        try:
            this_paper_type = data['root']['items']['item'][0]['@typeCode'] # так получается тип
        except:
            this_paper_type = 'none'
        author_types.append(this_paper_type) # добавляем тип статьи к списку всех типов статей аавтора
        
        ## ГОД статьи
        try:
            this_paper_year = data['root']['items']['item'][0]['@yearPubl']
        except:
            this_paper_year = 'none'
        author_years.append(this_paper_year) # добавляем год статьи к списку всех годов статей автора
        
        ## RETRACTED или нет?
        try:
            this_paper_retracted = data['root']['items']['item'][0]['@retracted'] # так получается статус (отозвана?)
        except:
            this_paper_retracted = 'none'
        author_retracteds.append(this_paper_retracted) # добавляем статус стаатьи (retracted или нет) к списку со статусами всех статей автора
        
        ## ТЕКСТ (не знаю что это означает) статьи
        try:
            this_paper_text = data['root']['items']['item'][0]['#text']
        except:
            this_paper_text = 'none'
        author_texts.append(this_paper_text) # добавляем текст статьи к списку с текстами (still dont know what does it means) статей автора
            
    author_personal_id = [str(author_id) for i in range(len(data['root']['items']['item']))]

    # получаем датафрейм где только одна строчка - данные по одной статье одного автора
    one_author = pd.DataFrame([author_personal_id,
              author_paper_ids,
              author_types,
              author_years,
              author_retracteds,
              author_texts]).transpose() 
            
    # теперь обновляем наш датафрейм где данные в формате одна строчка - один автор        
    one_row_one_paper = one_row_one_paper.append(one_author)
    
    
            
    counter += 1 #обновляем счетчик
    if counter % 50 == 0: # каждые 50 итерациий
        sleep(1) # засыпаем на 1 секунду

100%|██████████| 5/5 [00:00<00:00,  7.14it/s]


In [63]:
one_row_one_paper.columns = ['author_id',
                             'paper_id',
                             'paper_type',
                             'paper_year',
                             'paper_retracted',
                             'paper_text']

In [67]:
one_row_one_paper

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
181,651,20473633,COR,1966,0,30945032
182,651,21896365,COR,1966,0,30945032
183,651,21897522,COR,1966,0,30945032
184,651,25557109,COR,1966,0,30945032


## Основная выгрузка по всем 19 723 авторам: процесс остановился на 65% из-за проблем с  Wi-Fi сетью "HSE WORK"

In [ ]:
counter = 0

one_row_one_paper = pd.DataFrame()

for j in tqdm.tqdm(author_ids):
    author_id = j
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid="+str(j)
    response = requests.get(url)
    data = xmltodict.parse(response.content) # с этим объектом будем работать

    
    # создаем объекты с которыми будем работать, это все для записи данных по одному автору
    author_paper_ids = [] # айдишники статей одного автора
    author_types = [] # тип статей одного автора
    author_years = [] # год статей одного автора
    author_retracteds = [] # статус (отозвана или нет(?)) одного автора
    author_texts = [] # текст чего-то там одного автора
    
    # идем по всем статьям автора
    for i in range(len(data['root']['items']['item'])):
        
        ## ID статьи
        try:
            this_paper_id = data['root']['items']['item'][i]['@id']
        except:
            this_paper_id = 'none'
        author_paper_ids.append(this_paper_id) # добавляем id статьи к списку всех статей автора
        
        ## ТИП статьи 
        try:
            this_paper_type = data['root']['items']['item'][0]['@typeCode'] # так получается тип
        except:
            this_paper_type = 'none'
        author_types.append(this_paper_type) # добавляем тип статьи к списку всех типов статей аавтора
        
        ## ГОД статьи
        try:
            this_paper_year = data['root']['items']['item'][0]['@yearPubl']
        except:
            this_paper_year = 'none'
        author_years.append(this_paper_year) # добавляем год статьи к списку всех годов статей автора
        
        ## RETRACTED или нет?
        try:
            this_paper_retracted = data['root']['items']['item'][0]['@retracted'] # так получается статус (отозвана?)
        except:
            this_paper_retracted = 'none'
        author_retracteds.append(this_paper_retracted) # добавляем статус стаатьи (retracted или нет) к списку со статусами всех статей автора
        
        ## ТЕКСТ (не знаю что это означает) статьи
        try:
            this_paper_text = data['root']['items']['item'][0]['#text']
        except:
            this_paper_text = 'none'
        author_texts.append(this_paper_text) # добавляем текст статьи к списку с текстами (still dont know what does it means) статей автора
            
    author_personal_id = [str(author_id) for i in range(len(data['root']['items']['item']))]

    # получаем датафрейм где только одна строчка - данные по одной статье одного автора
    one_author = pd.DataFrame([author_personal_id,
              author_paper_ids,
              author_types,
              author_years,
              author_retracteds,
              author_texts]).transpose() 
            
    # теперь обновляем наш датафрейм где данные в формате одна строчка - один автор        
    one_row_one_paper = one_row_one_paper.append(one_author)
    
    counter += 1 #обновляем счетчик
    if counter % 50 == 0: # каждые 50 итерациий
        sleep(1) # засыпаем на 1 секунду

In [70]:
one_row_one_paper

,0,1,2,3,4,5
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
20,746161,23217910,RAR,1998,0,8323673
21,746161,24831641,RAR,1998,0,8323673
22,746161,30339514,RAR,1998,0,8323673
23,746161,30682370,RAR,1998,0,8323673


In [71]:
one_row_one_paper.columns = ['author_id',
                             'paper_id',
                             'paper_type',
                             'paper_year',
                             'paper_retracted',
                             'paper_text']

In [72]:
one_row_one_paper # за первый заход, данные получились такие

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
20,746161,23217910,RAR,1998,0,8323673
21,746161,24831641,RAR,1998,0,8323673
22,746161,30339514,RAR,1998,0,8323673
23,746161,30682370,RAR,1998,0,8323673


In [73]:
one_row_one_paper.to_excel('935755_rows_paper_ids_by_author_5_10.xlsx') # сохраняем в формате .xlsx
one_row_one_paper.to_csv('935755_rows_paper_ids_by_author_5_10.csv') # сохраняем в формате .csv

## Второй заход к довыгрузке данных

К сожалению, из-за проблем с сетью (сеть иногда просто берет и отключается) не получилось загрузить сразу все данные, а еще у меня перестало работать ядро. Я сохранила ноутбук, и перезашла.

In [4]:
one_row_one_paper = pd.read_csv('935755_rows_paper_ids_by_author_5_10.csv') # восстанавливаю загруженные ранее данные
one_row_one_paper.drop('Unnamed: 0', axis=1, inplace=True) # бросаю первую колонку с индексом строчек

In [5]:
one_row_one_paper # теперь данные выглядят так, как будто бы ничего и не произошло..

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
935750,746161,23217910,RAR,1998,0,8323673
935751,746161,24831641,RAR,1998,0,8323673
935752,746161,30339514,RAR,1998,0,8323673
935753,746161,30682370,RAR,1998,0,8323673


In [6]:
first_part = one_row_one_paper # сохраняем в переменную first_part, чтобы дальше не запутаться
first_part

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
935750,746161,23217910,RAR,1998,0,8323673
935751,746161,24831641,RAR,1998,0,8323673
935752,746161,30339514,RAR,1998,0,8323673
935753,746161,30682370,RAR,1998,0,8323673


### Выбираем необработанные id авторов

In [15]:
prepared_author_ids = list(np.unique(list(one_row_one_paper['author_id']))) # уточняем обработанные данные
print(f'Обработана информация по числу авторов: {len(prepared_author_ids)}')
print(f'Осталось обработать {len(author_ids) - len(prepared_author_ids)} авторов')

Обработана информация по числу авторов: 12789
Осталось обработать 6934 авторов


In [16]:
needed_author_ids = [i for i in author_ids if i not in prepared_author_ids]
print(len(needed_author_ids) == (len(author_ids) - len(prepared_author_ids))) # логическая проверка - да, все ок

True


In [17]:
needed_author_ids[0:10] # в этом объекте лежат id авторов, статьи по которым не были выгружены, 
# это первые 10 авторов

[746198,
 746247,
 746259,
 746281,
 746295,
 746386,
 746415,
 746425,
 746459,
 746466]

### Довыгрузка: статьи 6934 авторов

In [19]:
counter = 0 # обновляем счетчик

second_part = pd.DataFrame() # датафрейм для второй части

for j in tqdm.tqdm(needed_author_ids): # меняем author_ids на needed_author_ids
    author_id = j
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid="+str(j)
    response = requests.get(url)
    data = xmltodict.parse(response.content) # с этим объектом будем работать

    
    # создаем объекты с которыми будем работать, это все для записи данных по одному автору
    author_paper_ids = [] # айдишники статей одного автора
    author_types = [] # тип статей одного автора
    author_years = [] # год статей одного автора
    author_retracteds = [] # статус (отозвана или нет(?)) одного автора
    author_texts = [] # текст чего-то там одного автора
    
    # идем по всем статьям автора
    for i in range(len(data['root']['items']['item'])):
        
        ## ID статьи
        try:
            this_paper_id = data['root']['items']['item'][i]['@id']
        except:
            this_paper_id = 'none'
        author_paper_ids.append(this_paper_id) # добавляем id статьи к списку всех статей автора
        
        ## ТИП статьи 
        try:
            this_paper_type = data['root']['items']['item'][0]['@typeCode'] # так получается тип
        except:
            this_paper_type = 'none'
        author_types.append(this_paper_type) # добавляем тип статьи к списку всех типов статей аавтора
        
        ## ГОД статьи
        try:
            this_paper_year = data['root']['items']['item'][0]['@yearPubl']
        except:
            this_paper_year = 'none'
        author_years.append(this_paper_year) # добавляем год статьи к списку всех годов статей автора
        
        ## RETRACTED или нет?
        try:
            this_paper_retracted = data['root']['items']['item'][0]['@retracted'] # так получается статус (отозвана?)
        except:
            this_paper_retracted = 'none'
        author_retracteds.append(this_paper_retracted) # добавляем статус стаатьи (retracted или нет) к списку со статусами всех статей автора
        
        ## ТЕКСТ (не знаю что это означает) статьи
        try:
            this_paper_text = data['root']['items']['item'][0]['#text']
        except:
            this_paper_text = 'none'
        author_texts.append(this_paper_text) # добавляем текст статьи к списку с текстами (still dont know what does it means) статей автора
            
    author_personal_id = [str(author_id) for i in range(len(data['root']['items']['item']))]

    # получаем датафрейм где только одна строчка - данные по одной статье одного автора
    one_author = pd.DataFrame([author_personal_id,
              author_paper_ids,
              author_types,
              author_years,
              author_retracteds,
              author_texts]).transpose() 
            
    # теперь обновляем наш датафрейм где данные в формате одна строчка - один автор        
    second_part = second_part.append(one_author)
    
    counter += 1 #обновляем счетчик
    if counter % 50 == 0: # каждые 50 итерациий
        sleep(1) # засыпаем на 1 секунду

100%|██████████| 6934/6934 [15:29<00:00,  7.46it/s]  


Смотрим на данные.

In [18]:
first_part # первая часть

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
935750,746161,23217910,RAR,1998,0,8323673
935751,746161,24831641,RAR,1998,0,8323673
935752,746161,30339514,RAR,1998,0,8323673
935753,746161,30682370,RAR,1998,0,8323673


In [20]:
second_part # вторая часть

,0,1,2,3,4,5
0,746198,15900143,AUT,2006,0,15900143
1,746198,16084933,AUT,2006,0,15900143
2,746198,21467087,AUT,2006,0,15900143
3,746198,23941529,AUT,2006,0,15900143
4,746198,19880660,AUT,2006,0,15900143
...,...,...,...,...,...,...
6,1158931,43149292,PRC,2016,0,27464636
7,1158931,46203919,PRC,2016,0,27464636
8,1158931,46515617,PRC,2016,0,27464636
0,1158945,36736120,RAR,2018,0,36736120


In [21]:
first_part.to_excel('first_part.xlsx') # на всякий случай сразу же сохраняем данные
second_part.to_excel('second_part.xlsx') # и вторую часть тоже

In [23]:
second_part.columns = ['author_id', 'paper_id', 'paper_type', 'paper_year', 'paper_retracted',
       'paper_text'] # правильно называем колонки
second_part # смотрим на вторую часть

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,746198,15900143,AUT,2006,0,15900143
1,746198,16084933,AUT,2006,0,15900143
2,746198,21467087,AUT,2006,0,15900143
3,746198,23941529,AUT,2006,0,15900143
4,746198,19880660,AUT,2006,0,15900143
...,...,...,...,...,...,...
6,1158931,43149292,PRC,2016,0,27464636
7,1158931,46203919,PRC,2016,0,27464636
8,1158931,46515617,PRC,2016,0,27464636
0,1158945,36736120,RAR,2018,0,36736120


Получаем финальный файл (датафрейм `final`), содержащий в себе информацию о всех статьях авторов, которые были упомянуты в наших изначальных авторов

In [24]:
final = pd.concat([first_part, second_part])
final # финал

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
6,1158931,43149292,PRC,2016,0,27464636
7,1158931,46203919,PRC,2016,0,27464636
8,1158931,46515617,PRC,2016,0,27464636
0,1158945,36736120,RAR,2018,0,36736120


In [36]:
final['paper_year'].value_counts() # смотрим на количество статей по годам

2006    58829
2004    57999
2003    56808
2000    54151
2005    53293
        ...  
2021      119
1949      117
1955      113
1956       40
1931       11
Name: paper_year, Length: 70, dtype: int64

Далее, так как наши данные включают в себя (кажется) статьи, написанные в 2010 году и после, я создаю новую переменную `is_after_2010`, которая будет индицировать, была ли написана статья в 2010 году и позднее, или она была написана раньше

In [38]:
final['is_after_2010'] = final['paper_year'].apply(lambda x: 1 if x != 'none' and int(x) >= 2010 else 0)

In [40]:
final['is_after_2010'].value_counts() # фуф, всего 146 950 статей в 2010 году и после!

0    996555
1    146950
Name: is_after_2010, dtype: int64

In [46]:
only_after_2010 = final[final['is_after_2010'] == 1]
only_after_2010.drop(axis=1, labels='is_after_2010', inplace=True)
only_after_2010 # новый датафрейм

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
11688,4196,17111035,RAR,2011,0,17111035
11689,4196,26253152,RAR,2011,0,17111035
11690,4196,17977286,RAR,2011,0,17111035
11691,4196,18419257,RAR,2011,0,17111035
11692,4196,22817673,RAR,2011,0,17111035
...,...,...,...,...,...,...
6,1158931,43149292,PRC,2016,0,27464636
7,1158931,46203919,PRC,2016,0,27464636
8,1158931,46515617,PRC,2016,0,27464636
0,1158945,36736120,RAR,2018,0,36736120


In [48]:
only_after_2010.sort_values('paper_year', inplace=True) # для удобства сортирую данные по возрастанию по году

## Сохранение файлов

In [52]:
only_after_2010.to_excel('all_final_authors_with_papers_5_10_only_after_2010.xlsx', index=None)
only_after_2010.to_csv('all_final_authors_with_papers_5_10_only_after_2010.csv', index=None)

In [26]:
# final.to_excel('all_final_authors_with_papers_5_10.xlsx') excel-файл не сохраняется из-за размерности
final.to_csv('all_final_authors_with_papers_5_10.csv')